# SITCOM-989 - M1M3 Inertia Compensation Performance

We need plots and metrics to evaluate the performance of the M1M3 Inertia Compensation System as described in [SITCOM-989].    
Examples of plots are:

* Hardpoint Load Cell Forces Minima and Maxima during slews as a function of time.
* Correlate the plots above with accelerations, velocities, and positions.
* (any other ideas?)

Petr asked to analyse the data obtained when slewing the telescope around 80 deg in elevation with and without inertia forces. 
The two datasets below that he used as an example contain movement from -100 deg in azimuth to 100 deg in a single slew. 
On both cases, we are using 30% motion settings in azimuth. 

* [M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-02 22:02 - 2023-08-02 22:04 UTC]
* [M1M3 TMA Inertial forces Chronograph Dashboard on 2023-07-28 02:15 - 2023-07-28 02:17 UTC]

Added a new dataset containing similar data but with 50% azimuth motion settinds. 

* [M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-03 03:20 - 2023-08-03 03:22 UTC]

[SITCOM-989]: https://jira.lsstcorp.org/browse/SITCOM-989


[M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-02 22:02 - 2023-08-02 22:04 UTC]: https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=mean%28%29&lower=2023-08-02T20%3A00%3A00.000Z&upper=2023-08-03T02%3A00%3A00.000Z&zoomedLower=2023-08-02T22%3A02%3A24.799Z&zoomedUpper=2023-08-02T22%3A04%3A02.450Zhttps://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=mean%28%29&lower=2023-08-02T20%3A00%3A00.000Z&upper=2023-08-03T02%3A00%3A00.000Z&zoomedLower=2023-08-02T22%3A02%3A24.799Z&zoomedUpper=2023-08-02T22%3A04%3A02.450Z


[M1M3 TMA Inertial forces Chronograph Dashboard on 2023-07-28 02:15 - 2023-07-28 02:17 UTC]:https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=mean%28%29&lower=2023-07-28T02%3A00%3A00.000Z&upper=2023-07-28T03%3A30%3A00.000Z&zoomedLower=2023-07-28T02%3A15%3A45.730Z&zoomedUpper=2023-07-28T02%3A17%3A11.966Z

[M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-03 03:20 - 2023-08-03 03:22 UTC]:https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=5Hz&tempVars%5BFunction%5D=mean%28%29&lower=2023-08-03T03%3A20%3A00.000Z&upper=2023-08-03T03%3A22%3A00.000Z

## Notebook Preparation

In [ ]:
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from astropy.time import Time, TimeDelta
from datetime import timedelta
from scipy import integrate

from lsst.sitcom import vandv
from lsst.summit.utils.efdUtils import EfdClient, getDayObsForTime, makeEfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAEvent

In [ ]:
efd_client = makeEfdClient()

In [ ]:
number_of_hardpoints = 6
measured_forces_topics = [f"measuredForce{i}" for i in range(6)]

### Helper Functions and Classes

#### SlewDataSet

In [ ]:
class SlewDataSet:
    """
    Hold all the data associated with a slew.
    
    Parameters
    ----------
    name : str
        Human readable name
    begin : str or `astropy.time.Time`
        Start date and time in ISO format and in UTC.
    end : str or `astropy.time.Time`
        End date and time in ISO format and in UTC.
        
    Attributes
    ----------
    df : pd.DataFrame | None = None
        Table containing the HP measured forces (the columns should be 
        `measuredForces0`, `measuredForces1`, etc.). The index should be 
        a timestamp. 
    stats : pd.DataFrame | None = None
        Table containing the min/max/ptp values for each HP measured forces 
        in the beginning of a slew and at the end of a slew. 
    events : list[TMAEvent] | None = None
        TMA slew events obtained via TMAEventMaker to have a more exact 
        timestamp from MTMount.
    """
    def __init__(self, name : str, begin : str | Time, end : str | Time):
        self.name = name
        self.begin = begin if isinstance(begin, Time) else Time(begin, scale="utc", format="isot")
        self.end = end if isinstance(end, Time) else Time(end, scale="utc", format="isot")
        
    async def start(self):
        """Performs all the measurements"""
        self.df = await self.query_dataset()
        self.stats = self.get_stats()
        self.events = self.get_tma_slew_events()
        self.integrals = self.get_integral()
        
    async def query_dataset(self):
        """
        Queries all the relevant data, resample them to have the same requency and merge them in a single dataframe.
            
        Returns
        -------
        pd.DataFrame
        """    
        hp_measured_forces = await efd_client.select_time_series(
            "lsst.sal.MTM1M3.hardpointActuatorData", 
            measured_forces_topics, 
            Time(self.begin, scale='utc'), 
            Time(self.end, scale='utc')
        )
        return hp_measured_forces
    
    def get_stats(self):
        """
        Calculate statistics for each column in a given dataset.

        Returns
        -------
        pandas.DataFrame
            A DataFrame containing calculated statistics for each column in the dataset.
            For each column, the statistics include minimum, maximum, and peak-to-peak values.

        Notes
        -----
        This function computes statistics for each column in the provided dataset. It utilizes the `get_minmax` function
        to calculate minimum, maximum, and peak-to-peak values for each column's data.
        """
        stats = pd.DataFrame(
            data=[self.get_minmax(self.df[col]) for col in self.df.columns],
            index=self.df.columns
        )
        return stats
    
    @staticmethod
    def get_minmax(s):
        """
        Calculate minimum, maximum, and peak-to-peak values for two halves of a given pandas Series.

        Parameters
        ----------
        s : pandas.Series
            The input pandas Series containing data.

        Returns
        -------
        pandas.Series
            A Series containing the following calculated values for the two halves of the input Series:
            - begin_min: Minimum value of the first half of the Series.
            - begin_max: Maximum value of the first half of the Series.
            - begin_ptp: Peak-to-peak (ptp) value of the first half of the Series (abs(max - min)).
            - end_min: Minimum value of the second half of the Series.
            - end_max: Maximum value of the second half of the Series.
            - end_ptp: Peak-to-peak (ptp) value of the second half of the Series (abs(max - min)).

        Notes
        -----
        This function divides the input Series into two halves and calculates the minimum, maximum, and peak-to-peak values
        for each half. The peak-to-peak value is the absolute difference between the maximum and minimum values.

        Example
        -------
        >>> import pandas as pd
        >>> data = [1, 2, 3, 4, 5, 6, 7, 8]
        >>> series = pd.Series(data, name='example')
        >>> minmax_values = get_minmax(series)
        """

        half_timestamp = len(s.index) // 2
        first_half = s[:s.index[half_timestamp]]
        second_half = s[s.index[half_timestamp:]]

        first_half_ptp = abs(first_half.max() - first_half.min())
        second_half_ptp = abs(second_half.max() - second_half.min())

        result = pd.Series(
            data=[
                first_half.min(),
                first_half.max(),
                first_half_ptp,
                second_half.min(),
                second_half.max(),
                second_half_ptp,
            ],
            index=[
                "begin_min",
                "begin_max",
                "begin_ptp",
                "end_min",
                "end_max",
                "end_ptp",
            ],
            name=s.name,
        )

        return result
    
    def get_tma_slew_events(self, t_pad : int = 1):
        """
        Retrieve Telescope Mount Assembly (TMA) slew events within a specified time range.

        Parameters
        ----------
        t_pad : int, default=1
            Time padding in seconds to be added at the beginning and at the end of a slew event.

        Returns
        -------
        list
            A list of TMA slew events that occurred within the specified time range.

        Notes
        -----
        This function retrieves TMA slew events occurring between the specified start and end times.
        It uses the TMAEventMaker class to obtain events for the specified day of observation (dayObs).
        The events are filtered to include only those that start after 1 second before the specified start time
        and end before 1 second after the specified end time.

        Example
        -------
        >>> start_time = "2023-08-09T00:00:00"
        >>> end_time = "2023-08-09T23:59:59"
        >>> tma_events = get_tma_slew_events(start_time, end_time)
        """
        dayObs = getDayObsForTime(self.begin)
        print(self.begin, dayObs)

        eventMaker = TMAEventMaker()
        events = eventMaker.getEvents(dayObs)

        events = [e for e in events if (Time(e.begin, scale="utc") > self.begin - TimeDelta(t_pad, format="sec"))
                  & ((Time(e.end, scale="utc") < self.end + TimeDelta(t_pad, format="sec")))]

        return events
    
    def get_integral(self, t_pad : int = 1.):
    
        # Slice the region far from the beginning and the end of a slew
        evt = self.events[0]
        begin = pd.to_datetime(evt.begin.datetime + timedelta(seconds=t_pad), utc=True)
        end = pd.to_datetime(evt.end.datetime - timedelta(seconds=t_pad), utc=True)
        df = self.df[begin:end]

        # Convert pd.DateTimeIndex to a timestamp in seconds
        numeric_timestamp = df.index.astype(np.int64) / 1e9

        # Calculate the integral using different methods    
        standard_sum = [df[col].sum() for col in df]
        normalized_sum = [df[col].sum() / abs(numeric_timestamp[-1] - numeric_timestamp[0]) for col in df]
        simps_integration = [integrate.simps(df[col], numeric_timestamp) for col in df] 
        trapz_integration = [integrate.trapz(df[col], numeric_timestamp) for col in df]

        # Get Average, Mode and Std within the "stable" area
        mean = [df[col].mean() for col in df]
        median = [df[col].median() for col in df]
        std = [df[col].std() for col in df]

        integrals = pd.DataFrame(
            data=np.array([
                standard_sum, 
                normalized_sum, 
                simps_integration, 
                trapz_integration,
                mean,
                median,
                std
            ]).T,
            columns=[
                "standard_sum",
                "normalized_sum",
                "simps_integration",
                "trapz_integration",
                "mean",
                "median",
                "std"
            ],
            index=df.columns,
        )

        return integrals

## Data Collection

In [ ]:
# Dataset 1 - Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation On
begin = "2023-08-02T22:02:30"
end = "2023-08-02T22:04:00"

dataset1 = SlewDataSet(
    name="Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation On",
    begin=begin, 
    end=end,
)

await dataset1.start()

In [ ]:
# Dataset 2 - Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation On
begin = "2023-07-28T02:17:15" 
end = "2023-07-28T02:17:55"

dataset2 = SlewDataSet(
    name="Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation Off",
    begin=begin, 
    end=end,
)

await dataset2.start()

In [ ]:
# Dataset 3 - Az Slew from -100 to 100 with 80 El, 50% Az motion settings and Inertia Compensation On
begin = "2023-08-03T03:20:30"
end = "2023-08-03T03:21:20"

dataset3 = SlewDataSet(
    name="Az Slew from -100 to 100 with 80 El, Az 50% / El 30% motion settings and Inertia Compensation On",
    begin=begin, 
    end=end,
)

await dataset3.start()

## Plots and Analysis

In [ ]:
def plot_hp_measured_data(dataset):
    """
    Plots the HP Measured Data as a function of the time.
    
    Parameters
    ----------
    dataset : SlewDataSet
        Dataset containing the timestamps associated with the beginning and the
        end of a slew.
    """                   
    fig, ax = plt.subplots(
        num=f"HP Measured Forces - {dataset.name}",
        dpi=120,
        figsize=(12, 3)
    )

    
    for hp in range(number_of_hardpoints):
        topic = measured_forces_topics[hp]

        ax.plot(
            dataset.df[topic], 
            "-",
            label=f"HP{hp+1}",
            lw=0.5,
        )
        
    if dataset.events:
        t_pad = timedelta(seconds=5)
        for e in dataset.events:
            l_slew = ax.axvspan(Time(e.begin, scale="utc").datetime, Time(e.end, scale="utc").datetime, ec=None, fc='k', alpha=0.1, label="Slew Window", zorder=-1)
            l_integ = ax.axvspan(Time(e.begin, scale="utc").datetime + t_pad, Time(e.end, scale="utc").datetime - t_pad, ec=None, fc='b', alpha=0.1, label="Integration Window", zorder=-1)

    t_fmt = "%Y%m%d %H:%M:%S"
    ax.set_title(f"HP Measured Data\n {dataset.df.index[0].strftime(t_fmt)} - {dataset.df.index[-1].strftime(t_fmt)}")
    ax.set_xlabel("Time [UTC]")
    ax.set_ylabel("HP Measured Forces [N]")
    ax.grid(":", alpha=0.2)
    ax.legend(ncol=4)

    plt.show()

In [ ]:
%matplotlib inline

In [ ]:
plot_hp_measured_data(dataset1)
plot_hp_measured_data(dataset2)
plot_hp_measured_data(dataset3)

In [ ]:
print(dataset1.name)
dataset1.stats

In [ ]:
print(dataset2.name)
dataset2.stats

In [ ]:
print(dataset3.name)
dataset3.stats

In [ ]:
print(dataset1.name)
dataset1.integrals

In [ ]:
print(dataset2.name)
dataset2.integrals

In [ ]:
print(dataset3.name)
dataset3.integrals